# An Overview of Pandas Pivot Tables

In [4]:
import numpy as np
import pandas as pd

This exercise is inspired by Wes McKinney's [Python for Data Analysis](http://proquest.safaribooksonline.com.libproxy.berkeley.edu/book/programming/python/9781491957653)

In [5]:
df = pd.read_csv("elections.csv")
df.head()

Candidate        Party     %  Year Result
0    Reagan   Republican  50.7  1980    win
1    Carter   Democratic  41.0  1980   loss
2  Anderson  Independent   6.6  1980   loss
3    Reagan   Republican  58.8  1984    win
4   Mondale   Democratic  37.6  1984   loss

## Pivot Tables

Recall from before that we were able to group the % Series by the "Party" and "Result" Series, allowing us to understand the average vote earned by each party under each election result.

In [6]:
percent_grouped_by_party_and_result = df['%'].groupby([df['Party'], df['Result']])
percent_grouped_by_party_and_result.mean()

Party        Result
Democratic   loss      44.850000
             win       49.050000
Independent  loss      11.300000
Republican   loss      42.750000
             win       51.266667
Name: %, dtype: float64

Because we called `groupby` on a Series, the result of our aggregation operation was also a Series. However, I believe this data is more naturally expressed in a tabular format, with Party as the rows, and Result as the columns. The `pivot_table` operation is the natural way to achieve this data format.

In [7]:
df_pivot = df.pivot_table(
    index='Party', # the rows (turned into index)
    columns='Result', # the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot.head()

Result        loss        win
Party                        
Democratic   44.85  49.050000
Independent  11.30        NaN
Republican   42.75  51.266667

The basic idea is that you specify a Series to be the `index` (i.e. rows) and a Series to be the `columns`. The data in the specified `values` is then grouped by all possible combinations of values that occur in the `index` and `columns` Series. These groups are then aggregated using the `aggfunc`, and arranged into a table that matches the requested `index` and `columns`. The diagram below summarizes how pivot tables are formed. (Diagram inspired by Joey Gonzales). Diagram source at [this link](https://docs.google.com/presentation/d/1FrYg6yd6B-CIgfWLWm4W8vBhfmJ6Qt9dKkN-mlN5AKU/edit#slide=id.g4131093782_0_89).

![groupby](pivot_table_overview.png)

For more on pivot tables, see [this excellent tutorial](http://pbpython.com/pandas-pivot-table-explained.html) by Chris Moffitt.

## List Arguments to pivot_table (Extra)

The arguments to our pivot_table method can also be lists. A few examples are given below.

If we pivot such that only our `columns` argument is a list, we end up with columns that are MultiIndexed.

In [9]:
df_pivot = df.pivot_table(
    index='Party', # the rows (turned into index)
    columns=['Result', 'Candidate'], # the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot.head()

Result          loss                                                        \
Candidate   Anderson  Bush Carter Clinton  Dole Dukakis  Gore Kerry McCain   
Party                                                                        
Democratic       NaN   NaN   41.0    48.2   NaN    45.6  48.4  48.3    NaN   
Independent      6.6   NaN    NaN     NaN   NaN     NaN   NaN   NaN    NaN   
Republican       NaN  37.4    NaN     NaN  40.7     NaN   NaN   NaN   45.7   

Result                                   win                             
Candidate   Mondale  Perot Romney       Bush Clinton Obama Reagan Trump  
Party                                                                    
Democratic     37.6    NaN    NaN        NaN    46.1  52.0    NaN   NaN  
Independent     NaN  13.65    NaN        NaN     NaN   NaN    NaN   NaN  
Republican      NaN    NaN   47.2  50.666667     NaN   NaN  54.75  46.1

If we pivot such that only our `index` argument is a list, we end up with rows that are MultiIndexed.

In [21]:
df_pivot = df.pivot_table(
    index=['Party', 'Candidate'], # the rows (turned into index)
    columns='Result',# the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot

Result                  loss        win
Party       Candidate                  
Democratic  Carter     41.00        NaN
            Clinton    48.20  46.100000
            Dukakis    45.60        NaN
            Gore       48.40        NaN
            Kerry      48.30        NaN
            Mondale    37.60        NaN
            Obama        NaN  52.000000
Independent Anderson    6.60        NaN
            Perot      13.65        NaN
Republican  Bush       37.40  50.666667
            Dole       40.70        NaN
            McCain     45.70        NaN
            Reagan       NaN  54.750000
            Romney     47.20        NaN
            Trump        NaN  46.100000

If we pivot such that only our values argument is a list, then we again get a DataFrame with multi-indexed Columns.

In [24]:
df_pivot = df.pivot_table(
    index='Party', # the rows (turned into index)
    columns='Result',# the column values
    values=['%', 'Year'], # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot

%                    Year             
Result        loss        win         loss          win
Party                                                  
Democratic   44.85  49.050000  1995.333333  2002.000000
Independent  11.30        NaN  1989.333333          NaN
Republican   42.75  51.266667  2002.000000  1995.333333

Feel free to experiment with other possibilities!